# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20901999


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:03<00:44,  1.53s/it]

Rendering models:  10%|▉         | 3/31 [00:04<00:44,  1.58s/it]

Rendering models:  13%|█▎        | 4/31 [00:05<00:33,  1.23s/it]

Rendering models:  16%|█▌        | 5/31 [00:05<00:25,  1.02it/s]

...

Rendering models:  90%|█████████ | 28/31 [00:15<00:01,  2.08it/s]

Rendering models:  94%|█████████▎| 29/31 [00:16<00:00,  2.33it/s]

Rendering models:  97%|█████████▋| 30/31 [00:16<00:00,  2.55it/s]

Rendering models: 100%|██████████| 31/31 [00:16<00:00,  2.72it/s]

equidad1                              0.000080
taisribeiro                           0.001166
not-logged-in-70f9f0731e13b4023213    0.141458
ElisabethB                            0.000151
ravendrop                             0.000070
cmk24                                 0.000047
not-logged-in-e5409037b2fb11f8e397    0.015398
not-logged-in-61c60101aa7fea3372e3    0.000074
start056                              0.000108
smilingsloth.                         0.003584
TheoBe                                0.000078
coyotes54                             0.000078
gabrielweiss                          0.001376
clairedeu                             0.000074
pangeli5                              0.000094
not-logged-in-20a090b00932d22636ca    0.000146
acapirala                             0.000078
Joir                                  0.000219
alexmnelson18                         0.000089
Lavadude                              0.012972
not-logged-in-d1a2200e729ec53382b7    0.000072
pikachu_r    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())